In [1]:
!pip install pytesseract transformers sentence-transformers faiss-cpu chromadb openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.3 MB/s eta 0

In [37]:
!pip install langchain openai faiss-cpu tiktoken python-dotenv pypdf langchain-community langchain_openai
!pip install langchain-google-genai
!pip install groq langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 6.7 MB/s eta 0:00:00


In [38]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import numpy as np
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import pandas as pd
from google.colab import userdata
import os
from langchain_groq import ChatGroq
import re

In [39]:
groq_api_key = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = groq_api_key

In [40]:
import os

# Define the directory containing the bank screenshots
bankimages_dir = 'bankimages' # Assuming the folder is in the same directory as the notebook

extracted_texts_with_paths = []
original_text_snippets = [] # Store only the text for easy retrieval

# Check if the bankimages directory exists
if not os.path.exists(bankimages_dir):
    print(f"Error: The directory '{bankimages_dir}' was not found.")
else:
    # List all files in the directory and filter for image files (e.g., .jpg, .png)
    image_files = [f for f in os.listdir(bankimages_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]

    if not image_files:
        print(f"No image files found in the directory '{bankimages_dir}'.")
    else:
        for image_file in image_files:
            image_path = os.path.join(bankimages_dir, image_file)
            extracted_text = perform_ocr(image_path) # Use the perform_ocr function
            extracted_texts_with_paths.append({'path': image_path, 'text': extracted_text})
            original_text_snippets.append(extracted_text)

        print(f"Processed {len(extracted_texts_with_paths)} images and extracted text.")
        for item in extracted_texts_with_paths:
            print(f"Path: {item['path']}, Text: {item['text'][:50]}...") # Print first 50 chars

Processed 9 images and extracted text.
Path: bankimages/Login-Step1.jpg, Text: Welcome to HDFC Bank NetBanking
HDFC Bank

Login t...
Path: bankimages/Login-step3.jpg, Text: More authentication needed!

We at HEFC Bank take ...
Path: bankimages/Cards.jpg, Text:    

 

 

 

 

 

 

r. NetBanking Welcome. paRv...
Path: bankimages/login-step4.jpg, Text: More authentication needed!

Please note:

 

+. T...
Path: bankimages/AccountBalance.jpg, Text: NetBanking

Acenanl Susnerarys

Transact »
Enquire...
Path: bankimages/Login-Step2.jpg, Text: Welcome to HDFC Bank NetBanking
Hg niGiw

Login to...
Path: bankimages/FundTransfer.jpg, Text:   

NetBanking — wetcowe, panvarnata swaTea nga Pr...
Path: bankimages/HomePage.jpg, Text: Smnertiiey Penson Use: 10 asta Alets

 

Rankiny R...
Path: bankimages/AccountStatment.jpg, Text:  

 

NetBanking — wcccowe-reavaniata susania
si H...


In [43]:
#Apply Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=100)
documents = [Document(page_content=item['text']) for item in extracted_texts_with_paths] # Correct: creates a Document for each text snippet
texts = text_splitter.split_documents(documents)

 #4. create Vector DB
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert embeddings to numpy array of float32 before creating FAISS index
import numpy as np
text_embeddings_np = np.array([embeddings.embed_query(doc.page_content) for doc in texts]).astype('float32')

db_img_text = FAISS.from_documents(texts, embeddings)


llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
retriever = db_img_text.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
def Get_LLM_Response(question):
  response = qa_chain({"query": question})
  return response['result']

In [54]:
#question = "how to see Debit Card information"
#response = qa_chain({"query": question})
#print(response['result'])

In [49]:
!pip install gradio

In [61]:
import gradio as gr

def process_input(text_input):
  response = qa_chain({"query": text_input})
  result = response['result']
  return result

# Define a clear function to clear both input and output textboxes
def clear_inputs():
    return "", ""

'''
with gr.Blocks() as iface:
    gr.Markdown("<h1 style='text-align: center; background-color: #f0f0f0;'>FAQs</h1>") # Centered heading with background color
    with gr.Row():
        with gr.Column(): # Column for input
            text_input = gr.Textbox(lines=2, placeholder="Enter your text here", label="Input Text")
            submit_button = gr.Button("Submit")
            clear_button = gr.Button("Clear")
        with gr.Column(): # Column for output
            output = gr.Textbox(label="Output")
'''
with gr.Blocks() as iface:
    with gr.Row():
       gr.Markdown("<h1 style='text-align: center; background-color: #f0f0f0;'>FAQs</h1>") # Centered heading with background color
    with gr.Row():
        text_input = gr.Textbox(lines=2, placeholder="Enter your FAQ here", label="FAQ Text")
        output = gr.Textbox(label="Steps")
    with gr.Row():
        submit_button = gr.Button("Submit")
        clear_button = gr.Button("Clear")

    submit_button.click(fn=process_input, inputs=[text_input], outputs=output)
    clear_button.click(fn=clear_inputs, inputs=None, outputs=[text_input, output])


iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7817f9a3f3bb84eaed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
